In [1]:
from IPython.core.display import display, HTML
import warnings
display(HTML("<style>.container { width:100% !important; }</style>"))
#warnings.simplefilter("ignore")

# TODO

1. Add ability to dump trace to file.
2. Trace client-server interaction.
3. Add tracking convergence state to this notebook.
4. Move tracking convergence state to optimizer.
5. Separate ML-config from admin-config. 

# Goal

The goal of this notebook is to demonstrate the use of the Optimizer Evaluation Tools.

## Optimizer evaluation purpose

The goal of optimizer evaluation is to learn how to best match an optimizer configuration to an optimization problem.

## Optimizer evaluation strategy

Broadly, the optimizer evaluation strategy hinges on characterizing the optimizers' performance on a variety of optimization problems, under a variety of configurations. This should allow us to:
* find the strengths and limitations of the various optimizer configurations,
* discard the dominated configurations,
* ultimately match optimizer configurations with the problem.

Note: No absolute scale of optimzier performance seems to have garnered consensus in the community. Thus, in this exercise we will compare the optimizers with each other. We can also compare them to some published results, though it might be good to replicate them first.

A common way to compare multiple optimizers is to rank them for each problem given a particular budget (usually in terms of number of evaluations, though if we assume that the function is cheap to compute, optimizer resource consumption might be more relevant). Then we can report average rank across problems to compare the optimizers. Or we can do a matrix for each benchmark problem which optimizer dominates which.


### Optimizer performance characteristics

We aim to evaluate the following aspects of the optimizers' performance:
* convergence - is the optimizer finding the optima, More formally, can the optimizer get within epsilon of the known optimum at all?
* rate of convergence - how quickly is the optimizer finding the optima?
* trajectory of convergence - is the optimizer improving rapidly at first, and then reaching a plateau, is it climbing steadily, or does it stagnate for a long time, before finally shooting up?
* surrogate model goodness of fit - how well do the models fit the training data, validation data (out of bag samples), test data (observations gathered after the model was fit), random test data (random observations gathered after the model was fit). 
* computational cost - for now we can use the Tracer to capture runtime information, down the road we should monitor CPU, and memory utilization too.

### What to measure

* Convergence of the entire optimizer on synthetic and real data.
* Models' goodness of fit on synthetic and real data, in both on both guided and unguided observations.
* Utility function optimizers' performance on synthetic data.

We shall measure and plot all of the above metrics as a function of the number of observations that the optimzier has been fit on.

<hr>

##### Definition of optimum
Various definitions of optimum are possible, and we have built consensus that the user should choose a definition suitable for their purpose. Here are some options:
1. Best observation - simplest, but can be deceptive in presence of noise. 
2. Observations with the highest:
    1. predicted mean performance
    2. upper confidence bound on performance
    3. lower confidence bound on performance
3. Speculative optima - return configurations predicted by the surrogate model but not necessarily tested:
    1. maximum predicted mean
    2. maximum predicted upper confidence bound
    3. maximum predicted lower confidence bound
        
Once we graduate to multi-objective optimization, we will need to build pareto frontiers from the above.
<hr>

Note, that surrogate models' goodness of fit becomes very important for all but the first definition of optimum.

### Selecting the optimizer configuration

The ability of an optimizer to converge on an optimum is our fundamental requirement. From between the optimizers that can converge we can break ties using secondary criteria: rate of convergence, goodness of fit, computational complexity of the optimizer.

### Troubleshooting the optimizers

For the optimizers that do not converge, this framework should illuminate their modes of failure:
* Is the model not fitting the data well?
* Is the model fitting the data, but the exposed parameters don't affect performance, or is there too much noise?
* If the model has strong goodness of fit, and performance is sensitive to the parameters, is the utility function optimizer up to the job? 
* Are we using the right utility function?

## TODO: Model Evaluation

1. Perform model GoF evaluation on some functions out of the factory:
    * <s>Define some more configurations for the objective functions.</s>
    * <s>Create the objective function</s>
    * <s>Create the model</s>
    * <s>Feed the data to the model (one observationa at a time, then in batches)</s>
    * <s>Plot the GoF metrics as a function of number of iterations (repeat each experiment a few times).</s>
    * Define some more configurations for the surrogate model (num trees, refit frequency, etc.)

In [2]:
import pandas as pd

from mlos.OptimizerEvaluationTools.ObjectiveFunctionFactory import ObjectiveFunctionFactory
from mlos.OptimizerEvaluationTools.ObjectiveFunctionConfigStore import objective_function_config_store
#objective_function_config = objective_function_config_store.get_config_by_name('noisy_polynomial_objective')
objective_function_config = objective_function_config_store.default
objective_function = ObjectiveFunctionFactory.create_objective_function(objective_function_config)

# Let's make sure this thing works
#
random_params_df = objective_function.parameter_space.random_dataframe(num_samples=100)
values_df = objective_function.evaluate_dataframe(random_params_df)
combined_df = pd.concat([random_params_df, values_df], axis=1)
combined_df

x_0       x_1           y
0  -8.585654 -7.129854 -982.206869
1  -5.069919 -4.512110 -334.910719
2  -1.599666 -1.327131  -25.433867
3  -6.888565 -3.557636 -510.732455
4  -4.013404 -1.968689 -160.228538
..       ...       ...         ...
95 -4.080019 -4.947356 -260.442717
96 -2.675136 -2.146852  -78.411388
97 -5.811636 -6.874785 -547.193339
98 -7.429455 -8.735967 -916.164367
99 -3.859993 -1.373007 -138.883623

[100 rows x 3 columns]

In [3]:
import grpc
import mlos.global_values as global_values
from mlos.Grpc.OptimizerMicroserviceServer import OptimizerMicroserviceServer
from mlos.Optimizers.BayesianOptimizerFactory import BayesianOptimizerFactory
from mlos.Optimizers.BayesianOptimizerConfigStore import bayesian_optimizer_config_store
from mlos.Optimizers.OptimizationProblem import OptimizationProblem, Objective
from mlos.Tracer import Tracer

global_values.declare_singletons()
global_values.tracer = Tracer(actor_id="OptimizerEvaluationTools", thread_id=0)

# Let's stand up the Optimizer Microservice
#
server = OptimizerMicroserviceServer(port=50051, num_threads=10)
server.start()
optimizer_factory = BayesianOptimizerFactory(grpc_channel=grpc.insecure_channel('localhost:50051'))

In [4]:
from mlos.Optimizers.BayesianOptimizerConfigStore import bayesian_optimizer_config_store

optimizer_config = bayesian_optimizer_config_store.default
optimizer_config.experiment_designer_config.glow_worm_swarm_optimizer_config.num_iterations = 10
optimizer_config.min_samples_required_for_guided_design_of_experiments = 10

print(optimizer_config.to_json(indent=2))


# Let's instantiate the optimizer. 
#
optimizer = optimizer_factory.create_local_optimizer(
    optimizer_config=optimizer_config,
    optimization_problem=OptimizationProblem(
        parameter_space=objective_function.parameter_space,
        objective_space=objective_function.output_space,
        objectives=[Objective(name='y', minimize=True)]
    )
)

09/29/2020 01:57:01 -   BayesianOptimizerFactory -    INFO - [BayesianOptimizerFactory.py:  40 -    create_local_optimizer() ] Creating a bayesian optimizer with config: {"surrogate_model_implementation": "HomogeneousRandomForestRegressionModel", "experiment_designer_implementation": "ExperimentDesigner", "min_samples_required_for_guided_design_of_experiments": 10, "homogeneous_random_forest_regression_model_config.n_estimators": 10, "homogeneous_random_forest_regression_model_config.features_fraction_per_estimator": 1, "homogeneous_random_forest_regression_model_config.samples_fraction_per_estimator": 1, "homogeneous_random_forest_regression_model_config.regressor_implementation": "DecisionTreeRegressionModel", "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.criterion": "mse", "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.splitter": "best", "homogeneous_random_forest_regression_model_config.decision_tr

{
  "surrogate_model_implementation": "HomogeneousRandomForestRegressionModel",
  "experiment_designer_implementation": "ExperimentDesigner",
  "min_samples_required_for_guided_design_of_experiments": 10,
  "homogeneous_random_forest_regression_model_config.n_estimators": 10,
  "homogeneous_random_forest_regression_model_config.features_fraction_per_estimator": 1,
  "homogeneous_random_forest_regression_model_config.samples_fraction_per_estimator": 1,
  "homogeneous_random_forest_regression_model_config.regressor_implementation": "DecisionTreeRegressionModel",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.criterion": "mse",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.splitter": "best",
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.max_depth": 0,
  "homogeneous_random_forest_regression_model_config.decision_tree_regression_model_config.min_samples_spli

In [5]:
# Let us set up the lists to track optima over time.
#
best_observation_num_observations = []
best_observation_configs = []
best_observations = []

predicted_value_num_observations = []
best_predicted_value_configs = []
best_predicted_values = []

regression_model_fit_state = RegressionModelFitState()

In [6]:
logger = create_logger("Optimizer evaluation.")
i = 0

In [22]:
from mlos.Logger import create_logger
from mlos.Optimizers.OptimumDefinition import OptimumDefinition
from mlos.Optimizers.RegressionModels.GoodnessOfFitMetrics import GoodnessOfFitMetrics, DataSetType
from mlos.Optimizers.RegressionModels.RegressionModelFitState import RegressionModelFitState


start_iteration_num = i
end_iteration_num = start_iteration_num + 10

for i in range(start_iteration_num, end_iteration_num):
    parameters = optimizer.suggest()
    objectives = objective_function.evaluate_point(parameters)
    logger.info(f"[{i+1}/{end_iteration_num}]Parameters: {parameters}, objectives: {objectives}")
    
    optimizer.register(parameters.to_dataframe(), objectives.to_dataframe())
    
    try:
        gof_metrics = optimizer.compute_surrogate_model_goodness_of_fit()
        regression_model_fit_state.set_gof_metrics(data_set_type=DataSetType.TRAIN, gof_metrics=gof_metrics)
    except:
        self.logger.error("Failed to compute goodness of fit metrics.", exc_info=True)
        
    
    best_observation_num_observations.append(i)
    
    best_observation_config, best_observation = optimizer.optimum(OptimumDefinition.BEST_OBSERVATION)    
    best_observation_configs.append(best_observation_config)
    best_observations.append(best_observation)
    
    try:
        best_predicted_value_config, best_predicted_value = optimizer.optimum(OptimumDefinition.PREDICTED_VALUE_FOR_OBSERVED_CONFIG)
        best_predicted_value_configs.append(best_predicted_value_config)
        best_predicted_values.append(best_predicted_value)
        predicted_value_num_observations.append(i)
    except:
        pass
    

09/29/2020 02:00:22 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [7/1006]Parameters: {"x_0": -8.91749776673975, "x_1": -9.659471515591923}, objectives: {"y": -1260.6089824126732}
09/29/2020 02:00:22 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [7/1006]Parameters: {"x_0": -8.91749776673975, "x_1": -9.659471515591923}, objectives: {"y": -1260.6089824126732}
09/29/2020 02:00:22 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [7/1006]Parameters: {"x_0": -8.91749776673975, "x_1": -9.659471515591923}, objectives: {"y": -1260.6089824126732}
09/29/2020 02:00:22 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [8/1006]Parameters: {"x_0": -4.120547236477767, "x_1": -8.738356882623481}, objectives: {"y": -490.4845195093524}
09/29/2020 02:00:22 

09/29/2020 02:00:37 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [18/1006]Parameters: {"x_0": -9.52216703193636, "x_1": -8.725157676299222}, objectives: {"y": -1290.6477083103293}
09/29/2020 02:00:37 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [18/1006]Parameters: {"x_0": -9.52216703193636, "x_1": -8.725157676299222}, objectives: {"y": -1290.6477083103293}
09/29/2020 02:00:39 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [19/1006]Parameters: {"x_0": -9.873605214228087, "x_1": -7.312107215022257}, objectives: {"y": -1239.9971001024142}
09/29/2020 02:00:39 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [19/1006]Parameters: {"x_0": -9.873605214228087, "x_1": -7.312107215022257}, objectives: {"y": -1239.9971001024142}
09/29/2020 02:

09/29/2020 02:00:54 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [29/1006]Parameters: {"x_0": -9.848184014541443, "x_1": -9.676764103869939}, objectives: {"y": -1447.6702173788403}
09/29/2020 02:00:54 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [30/1006]Parameters: {"x_0": -2.3059744835127676, "x_1": -5.391632813912292}, objectives: {"y": -155.85358066087713}
09/29/2020 02:00:54 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [30/1006]Parameters: {"x_0": -2.3059744835127676, "x_1": -5.391632813912292}, objectives: {"y": -155.85358066087713}
09/29/2020 02:00:54 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [30/1006]Parameters: {"x_0": -2.3059744835127676, "x_1": -5.391632813912292}, objectives: {"y": -155.85358066087713}
09/29/202

09/29/2020 02:01:02 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [41/1006]Parameters: {"x_0": -6.328849863287402, "x_1": -9.69863294534123}, objectives: {"y": -834.1373367495512}
09/29/2020 02:01:02 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [41/1006]Parameters: {"x_0": -6.328849863287402, "x_1": -9.69863294534123}, objectives: {"y": -834.1373367495512}
09/29/2020 02:01:02 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [41/1006]Parameters: {"x_0": -6.328849863287402, "x_1": -9.69863294534123}, objectives: {"y": -834.1373367495512}
09/29/2020 02:01:02 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [42/1006]Parameters: {"x_0": -9.40314624431487, "x_1": -8.152132209795123}, objectives: {"y": -1216.5321123699746}
09/29/2020 02:01:02

09/29/2020 02:01:14 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [52/1006]Parameters: {"x_0": -9.865023633418637, "x_1": -9.13347896057845}, objectives: {"y": -1398.2086478653239}
09/29/2020 02:01:14 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [52/1006]Parameters: {"x_0": -9.865023633418637, "x_1": -9.13347896057845}, objectives: {"y": -1398.2086478653239}
09/29/2020 02:01:14 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [53/1006]Parameters: {"x_0": -9.952833039185393, "x_1": -2.2499982364020985}, objectives: {"y": -952.8258005602279}
09/29/2020 02:01:14 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [53/1006]Parameters: {"x_0": -9.952833039185393, "x_1": -2.2499982364020985}, objectives: {"y": -952.8258005602279}
09/29/2020 02:

...

09/29/2020 02:13:24 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [630/1006]Parameters: {"x_0": -1.2803352447566052, "x_1": -7.062061763849254}, objectives: {"y": -188.1066936339579}
09/29/2020 02:13:24 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [630/1006]Parameters: {"x_0": -1.2803352447566052, "x_1": -7.062061763849254}, objectives: {"y": -188.1066936339579}
09/29/2020 02:13:26 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [631/1006]Parameters: {"x_0": -9.863409488709994, "x_1": -9.463298920784055}, objectives: {"y": -1429.7400261990717}
09/29/2020 02:13:26 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [631/1006]Parameters: {"x_0": -9.863409488709994, "x_1": -9.463298920784055}, objectives: {"y": -1429.7400261990717}
09/29/20

09/29/2020 02:13:39 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [641/1006]Parameters: {"x_0": -9.95773569822669, "x_1": -8.920560981370048}, objectives: {"y": -1397.293154170376}
09/29/2020 02:13:40 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [642/1006]Parameters: {"x_0": -4.915972213167055, "x_1": -6.961469979016087}, objectives: {"y": -449.1339113122688}
09/29/2020 02:13:40 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [642/1006]Parameters: {"x_0": -4.915972213167055, "x_1": -6.961469979016087}, objectives: {"y": -449.1339113122688}
09/29/2020 02:13:40 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [642/1006]Parameters: {"x_0": -4.915972213167055, "x_1": -6.961469979016087}, objectives: {"y": -449.1339113122688}
09/29/2020 02

09/29/2020 02:13:56 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [653/1006]Parameters: {"x_0": -9.623053462883949, "x_1": -9.676875476318438}, objectives: {"y": -1401.345525554755}
09/29/2020 02:13:56 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [653/1006]Parameters: {"x_0": -9.623053462883949, "x_1": -9.676875476318438}, objectives: {"y": -1401.345525554755}
09/29/2020 02:13:56 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [653/1006]Parameters: {"x_0": -9.623053462883949, "x_1": -9.676875476318438}, objectives: {"y": -1401.345525554755}
09/29/2020 02:13:57 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [654/1006]Parameters: {"x_0": -1.5125100519434422, "x_1": -4.586594144249513}, objectives: {"y": -89.30503821166383}
09/29/2020 

09/29/2020 02:14:11 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [664/1006]Parameters: {"x_0": -5.379353528158268, "x_1": -9.251854072790158}, objectives: {"y": -669.8920827491196}
09/29/2020 02:14:11 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [664/1006]Parameters: {"x_0": -5.379353528158268, "x_1": -9.251854072790158}, objectives: {"y": -669.8920827491196}
09/29/2020 02:14:14 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [665/1006]Parameters: {"x_0": -9.570068088969624, "x_1": -9.776162090665869}, objectives: {"y": -1400.4339631178916}
09/29/2020 02:14:14 -      Optimizer evaluation. -    INFO - [<ipython-input-22-3f39c58cff56>:  11 -                  <module>() ] [665/1006]Parameters: {"x_0": -9.570068088969624, "x_1": -9.776162090665869}, objectives: {"y": -1400.4339631178916}
09/29/2020

KeyboardInterrupt: 

In [23]:
# Best observation dataframe
#
best_observation_df = pd.DataFrame([observation.to_dict() for observation in best_observations])
best_observation_df['num_observations'] = best_observation_num_observations
best_observation_df = pd.concat([best_observation_df.drop_duplicates(subset=['y'], keep='last'), best_observation_df.drop_duplicates(subset=['y'], keep='first')]).sort_index()
best_observation_df

y  num_observations
0    -962.431708                 0
9    -962.431708                 9
10  -1097.227593                 9
16  -1097.227593                15
17  -1150.510006                16
40  -1150.510006                38
41  -1185.281066                39
42  -1185.281066                40
43  -1296.284034                41
52  -1296.284034                50
53  -1347.651315                51
54  -1347.651315                52
55  -1428.515070                53
55  -1428.515070                53
56  -1449.914938                54
85  -1449.914938                21
86  -1456.078868                22
137 -1456.078868                73
138 -1463.497507                74
173 -1463.497507               109
174 -1464.242288               110
214 -1464.242288               150
215 -1482.891984               151
226 -1482.891984               162
227 -1489.757319               163
239 -1489.757319               175
240 -1502.499890               176
320 -1502.499890               256
321 -1503.445350               257
493 -1503.445350               429
494 -1505.487549               430
532 -1505.487549               468
533 -1506.623589               469
737 -1506.623589               673

In [24]:
best_predicted_value_df = pd.DataFrame([predicted_value.to_dict() for predicted_value in best_predicted_values])
best_predicted_value_df['num_observations'] = predicted_value_num_observations
best_predicted_value_df = pd.concat([best_predicted_value_df.drop_duplicates(subset=['predicted_value'], keep='last'), best_predicted_value_df.drop_duplicates(subset=['predicted_value'], keep='first')]).sort_index()
best_predicted_value_df

predicted_value  num_observations
0        -835.962411                 9
9        -835.962411                18
10       -961.204762                18
19       -961.204762                27
20      -1050.426291                28
..               ...               ...
709     -1504.154067               655
710     -1504.918832               656
719     -1504.918832               665
720     -1505.256819               666
727     -1505.256819               673

[146 rows x 2 columns]

In [25]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
%matplotlib inline

fig, axs = plt.subplots(1, figsize=(11, 20), dpi=80, sharex=True)

axs.plot(best_observation_df['num_observations'], best_observation_df['y'], label='y')
axs.plot(best_predicted_value_df['num_observations'], best_predicted_value_df['predicted_value'], label='predicted_value')
axs.set_ylabel('y')
axs.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2f'))
axs.set_xticks(best_observation_df['num_observations'][::2])
axs.grid(True)
axs.set_xlabel('num_observations')
axs.legend()  
fig.show()

C:\Users\adsmiech\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  from ipykernel import kernelapp as app


In [19]:
from mlos.Optimizers.RegressionModels.GoodnessOfFitMetrics import DataSetType

# Let's take a look at goodness of fit data.
#
goodness_of_fit_dataframe = optimizer.get_surrogate_model_fit_state().get_goodness_of_fit_dataframe(data_set_type=DataSetType.TRAIN) # TODO: add support to evaluate GoF on test data
goodness_of_fit_dataframe

observation_count  prediction_count  mean_absolute_error  \
11                 11                11            90.530262   
21                 21                21            45.919828   
31                 31                31            42.208384   
41                 41                41            40.401525   
51                 51                51            29.113139   
61                 61                61            33.829471   

    root_mean_squared_error  relative_absolute_error  relative_squared_error  \
11               118.002944                 0.317768                0.338715   
21                69.599976                 0.155812                0.200265   
31                64.495981                 0.138063                0.185731   
41                64.090981                 0.136145                0.192666   
51                44.516450                 0.084628                0.111416   
61                51.886775                 0.096312                0.122798   

    coefficient_of_determination  prediction_90_ci_hit_rate  \
11                      0.885272                   1.000000   
21                      0.959894                   1.000000   
31                      0.965504                   0.935484   
41                      0.962880                   0.951220   
51                      0.987586                   1.000000   
61                      0.984921                   0.983607   

    sample_90_ci_hit_rate  
11                    1.0  
21                    1.0  
31                    1.0  
41                    1.0  
51                    1.0  
61                    1.0

In [20]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
%matplotlib inline

gof_df = goodness_of_fit_dataframe
columns_to_plot = [name for name in gof_df.columns.values if name not in ('observation_count', 'prediction_count')]
num_plots = len(columns_to_plot)
fig, axs = plt.subplots(num_plots, figsize=(11, 20), dpi=80, sharex=True)

for i, column in enumerate(columns_to_plot):
    axs[i].plot(gof_df.index, gof_df[column], marker='o', label=column)
    axs[i].set_ylabel(column)
    axs[i].yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2f'))
    axs[i].set_xticks(gof_df.index[::5])
    axs[i].grid(True)
    if i == num_plots - 1:
        axs[i].set_xlabel('observation_count')
        
fig.show()

C:\Users\adsmiech\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.


In [21]:
global_values.tracer.dump_trace_to_file(r"E:\code\new_mlos\source\Mlos.Python\temp\optimizer_evaluation_tools.json")

## TODO: Bayesian Optimizer Evaluation

1. Perform optimizer convergence evaluation on some functions out of the factory:
    1. Define some named configurations for the BayesianOptimizer (surrogate model config, utility function, utility function optimizer config).
    2. Write the optimum object to:
        1. Include all observations (at first - once we know what we don't need, we can remove them)
        2. Include surrogate model predictions for all observations.
        3. Include a dataframe with the various definitions of optimum:
            1. Best observed observation.
            2. Observation with the highest:
                1. predicted mean
                2. upper confidence bound
                3. lower confidence bound
            3. Speculative optima - kick of the utility function optimizer to find configurations with:
                1. maximum predicted mean
                2. maximum upper confidence bound
                3. maximum lower confidence bound
        1. Include a dataframe with cummax of all of the 7 optima.
        2. Compare the number of iterations needed to reach the same optimum for the various optimizers.
        
    3. Train the various models on the various functions and plot all 7 optima as a function of a number of observations (repeat each experiment a few times to get the idea of stability).
    4. Progressively increase the difficulty of the optimization problems:
        1. Amount of noise.
        2. Number of dimensions.
        3. Discontinuous functions.
        4. Search spaces with more branching and nesting.
        

## TODO: Utility Function Optimizer Evaluation

This is a two-step process.
1. Evaluate these model-free optimizers on the ObjectiveFunctionBase subclasses. This should allow us to capture any bugs, and get a sense of how quickly these optimizers converge on a variety of functions.
2. Evaluate these model-free optimizers as part of the bayesian optimizer. The key difference is that the model gets refit, so the underlying function changes between optimizer invocations, but we get to use prior invocations as starting points.

3. Plot all of this :)